In [1]:
import re
import requests
from requests.auth import HTTPBasicAuth
import json
import pandas as pd

In [ ]:
API_KEY = '-----------------------------------------' 
# 這邊使用的是線上的生成亂數帳號的 api，申請 api key 後填入就可以獲得。
# ref: https://api-ninjas.com/api/passwordgenerator

def get_password(length = '10'):
    password_url = f'https://api.api-ninjas.com/v1/passwordgenerator?length={length}&exclude_special_chars=true'
    response = requests.get(password_url, headers={'X-Api-Key': API_KEY})
    if response.status_code == requests.codes.ok:
        password = response.json()['random_password']
        return password
    else:
        print("Error:", response.status_code, response.text)

admin_username = "admin"
admin_password = "-------------------------" # 更改 admin password
auth = HTTPBasicAuth(admin_username, admin_password)

base_url = "http://10.217.33.12"
headers = {
    'Content-Type': 'application/json',
}    

def login(user, password = None):
    session = requests.Session()
    login_page = session.get(f"{base_url}/login", verify=False)
    csrf_token = re.search(r'<input type="hidden" name="_csrf_token" value="([^"]*)">', login_page.text).group(1)
    
    if not csrf_token:
        print(login_page.text)
    # Login
    payload = {
        "_username": user,
        "_password": password if password else user,
        "_csrf_token": csrf_token
    }

    login_response = session.post(f"{base_url}/login", data = payload, verify=False)

    if login_response.status_code == 200:
        print(f"{user} logged in successfully.")
    else:
        raise Exception(f"Failed to login, user: {user}.")

def add_user(data):
    response = requests.post(f"{base_url}/api/users", auth = auth, data = json.dumps(data), headers = headers)
    
    # Check the response
    if response.status_code in [200, 201]:
        pass
    else:
        raise Exception('Failed:', response.status_code, response.text)

def add_team(data, contest_id = "2"):
    response = requests.post(f"{base_url}/api/contests/{contest_id}/teams", auth = auth, data = json.dumps(data), headers = headers)
    
    # Check the response
    if response.status_code in [200, 201]:
        print('Success:', response.json())
        return response.json()
    else:
        raise Exception('Failed:', response.status_code, response.text)

login(admin_username, admin_password) # 確認 url 是正確連上的

In [ ]:
# 壓測創建帳號
trial = 1
batch = 250
member = 3

for i in range(1 + batch * (trial - 1), 1 + batch * trial):
    team = add_team({
        "icpc_id": f"test_team{i}",
        "label": f"test_team{i}",
        "display_name": f"test_team{i}",
        "name": f"test_team{i}",
        "id": f"test_team{i}",
        "group_ids": [3] # group 3 是 participant，如果重新設定 domjudge 可能會有變動
    })
    for j in range(1, member + 1):
        add_user(data = {
            "username": f"test_team{i}_user{j}",
            "name":f"test_team{i}_user{j}",
            "roles": ["team"],
            "password": str(j) * 10,
            "team_id": team["id"]
        })

In [ ]:
# 線上賽創建帳號

# users.xlsx format
"""
No ID	英文姓名	中文姓名	電話	郵件信箱	英文姓名	中文姓名	年紀	通過資格考	隊名
A018	Siah Yu Kyle	谢宇凯	601127808325	codsiah@gmail.com	 SMK(L) Methodist, Kuala Lumpur	 SMK(L) Methodist, Kuala Lumpur	4	yes	ralat logik
A020	Bryden Wong Jee Young	to trace	60106556317	wong.bryden@gmail.com	 SMK(L) Methodist, Kuala Lumpur	 SMK(L) Methodist, Kuala Lumpur	4	yes	汪汪队
"""
users_df = pd.read_excel("users.xlsx")
users_dict = users_df.groupby("隊名").apply(lambda x: x[['No ID', '郵件信箱']].values.tolist()).to_dict()

output_teams = []
output_emails = []
output_users = []
passwords = []

exception = []


for team_id, (team, users) in enumerate(users_dict.items(), 1):
    if len(users) != 3:
        exception.append({team: users})
        continue
        
    response = add_team({
        "icpc_id": f"team{team_id}",
        "label": team,
        "display_name": team,
        "name": team,
        "id": f"team{team_id}",
        "group_ids": [3]
    })
    
    for [user, email] in users:
        output_teams.append(team)
        output_users.append(user)
        output_emails.append(email)

        password = get_password()
        passwords.append(password)
        add_user(data = {
            "username": user,
            "name": user,
            "roles": ["team"],
            "password": password,
            "team_id": response["id"]
        })

In [ ]:
# 實體賽創建帳號
skip_users = [f"user{i}" for i in ["24", "49", "64", "01", "17", "58", "23", "69", "15", "70"]]

# final_new.csv format
"""
team,username,password,team_id
BLANK SPACE,user01,--------,A1
汪汪队,user02,--------,A2
"""
users_df = pd.read_csv("final_new.csv")
users_dict = users_df.groupby(['team', 'team_id'], sort=False).apply(lambda x: x[['username', 'password']].values.tolist()).to_dict()

for _, ((team, team_id), users) in enumerate(users_dict.items(), 517):
    if team_id != "backup":
        response = add_team({
            "icpc_id": team_id,
            "label": team,
            "display_name": team,
            "name": team,
            "id": team_id,
            "group_ids": [3]
        })
        for [user, password] in users:
            if user not in skip_users:
                add_user(data = {
                    "username": user,
                    "name": user,
                    "roles": ["team"],
                    "password": password,
                    "team_id": response["id"]
                })